In [ ]:
pip install --upgrade torch torchvision

In [ ]:
# https://drive.google.com/file/d/1S7upK1OsxQoLfOLDTR-Px3XJFGuZKn3B/view?usp=sharing
!gdown https://drive.google.com/uc?id=1S7upK1OsxQoLfOLDTR-Px3XJFGuZKn3B

In [ ]:
!rm -r TLDataset/ALL/PKG-C-NMC2019/train/.ipynb_checkpoints
!rm -r TLDataset/ALL/PKG-C-NMC2019/train/.DS_STORE

In [ ]:
from torchvision.models import ConvNeXt_Base_Weights

In [ ]:
!pip install seaborn

In [ ]:
pip install --upgrade efficientnet-pytorch

In [ ]:
!pip install torch-summary

In [ ]:
!pip install summary

# Training

In [14]:
# start training
!python train-vt.py --model_name vit_base_patch16_224 --data_dir TLDataset/ALL/PKG-C-NMC2019 --num_classes 2 --num_epochs 50 --batch_size 32 --learning_rate 0.001 --checkpoint_path model_checkpoints/ALL/vit/

Using device: cuda:0
Training Data: 7845 samples
Validation Data: 2347 samples
Layer (type:depth-idx)                   Output Shape              Param #
├─PatchEmbed: 1-1                        [-1, 196, 768]            --
|    └─Conv2d: 2-1                       [-1, 768, 14, 14]         590,592
|    └─Identity: 2-2                     [-1, 196, 768]            --
├─Dropout: 1-2                           [-1, 197, 768]            --
├─Identity: 1-3                          [-1, 197, 768]            --
├─Identity: 1-4                          [-1, 197, 768]            --
├─Sequential: 1-5                        [-1, 197, 768]            --
|    └─Block: 2-3                        [-1, 197, 768]            --
|    |    └─LayerNorm: 3-1               [-1, 197, 768]            1,536
|    |    └─Attention: 3-2               [-1, 197, 768]            2,362,368
|    |    └─Identity: 3-3                [-1, 197, 768]            --
|    |    └─Identity: 3-4                [-1, 197, 768]      

# Evaluation 

ALL Only Evaluation

In [15]:
!python evaluate_vt.py --model_name vit_base_patch16_224 --data_dir ALL_Unseen/ --model_path model_checkpoints/ALL/vit/vit_base_patch16_224_best.pth --num_classes 2 --batch_size 32


Evaluation Results:
Precision: 0.8158
Recall: 0.8147
F1: 0.8152
Accuracy: 0.8401
Balanced accuracy: 0.8147


# Inference

In [ ]:
!python inference.py --model_name resnet50 --image_path TLDataset/ALL/PKG-C-NMC2019/C-NMC_test_prelim_phase_data --model_path model_checkpoints/ALL/resnet50_best_epoch_49.pth --num_classes 2 --image_save results_C-NMC_test_prelim_phase_data

In [ ]:
!python inference.py --model_name resnet50 --image_path TLDataset/ALL/PKG-C-NMC2019/C-NMC_test_final_phase_data --model_path model_checkpoints/ALL/resnet50_best_epoch_49.pth --num_classes 2 --image_save results_C-NMC_test_final_phase_data

# Transfer learning 

Here the trained model on ALL dataset is finetuned for AML dataset, by loading the ALL model weights as pretrained model.

In [16]:
!python train-vt.py --model_name vit_base_patch16_224 --data_dir AMLDataset/AML --num_classes 2 --num_epochs 50 --batch_size 32 --learning_rate 0.001 --checkpoint_path model_checkpoints/AML/vit --pretrained_path model_checkpoints/ALL/vit/vit_base_patch16_224_best.pth

Using device: cuda:0
Training Data: 12207 samples
Validation Data: 5230 samples
Layer (type:depth-idx)                   Output Shape              Param #
├─PatchEmbed: 1-1                        [-1, 196, 768]            --
|    └─Conv2d: 2-1                       [-1, 768, 14, 14]         590,592
|    └─Identity: 2-2                     [-1, 196, 768]            --
├─Dropout: 1-2                           [-1, 197, 768]            --
├─Identity: 1-3                          [-1, 197, 768]            --
├─Identity: 1-4                          [-1, 197, 768]            --
├─Sequential: 1-5                        [-1, 197, 768]            --
|    └─Block: 2-3                        [-1, 197, 768]            --
|    |    └─LayerNorm: 3-1               [-1, 197, 768]            1,536
|    |    └─Attention: 3-2               [-1, 197, 768]            2,362,368
|    |    └─Identity: 3-3                [-1, 197, 768]            --
|    |    └─Identity: 3-4                [-1, 197, 768]     

# Notes

Inference on unseen AML Images

In [ ]:
# TL model with unseen ALL images again to see if TL helped

In [17]:
!python evaluate_vt.py --model_name vit_base_patch16_224 --data_dir ALL_Unseen/ --model_path model_checkpoints/AML/vit/vit_base_patch16_224_best.pth --num_classes 2 --batch_size 32

Evaluating: 100%|█████████████████████| 15/15 [00:01<00:00,  8.55it/s, acc=0.32]

Evaluation Results:
Precision: 0.4921
Recall: 0.4998
F1: 0.2468
Accuracy: 0.3198
Balanced accuracy: 0.4998


In [ ]:
# TL model with unseen AML images

In [18]:
!python evaluate_vt.py --model_name vit_base_patch16_224 --data_dir AML_Unseen/ --model_path model_checkpoints/AML/vit/vit_base_patch16_224_best.pth --num_classes 2 --batch_size 32

Evaluating: 100%|████████████████████| 29/29 [00:03<00:00,  7.54it/s, acc=0.957]

Evaluation Results:
Precision: 0.9352
Recall: 0.9298
F1: 0.9325
Accuracy: 0.9575
Balanced accuracy: 0.9298


In [ ]:
# Training AML Only without TF weights to Compare with TF Model

In [ ]:
!pip install seaborn

In [19]:
# start training AML Model
!python train-vt.py --model_name vit_base_patch16_224 --data_dir AMLDataset/AML/ --num_classes 2 --num_epochs 50 --batch_size 32 --learning_rate 0.001 --checkpoint_path model_checkpoints/AML_Only/vit/


Epoch [1/50], Train Loss: 0.5064, LR: 0.001000                                  
Validation: 100%|███████████| 164/164 [00:15<00:00, 10.83it/s, val_loss=0.00848]
Validation Loss: 0.0114, F1: 0.6030
Saved Best Model at epoch 1 with F1 score: 0.6030
Epoch [2/50], Train Loss: 0.2849, LR: 0.001000                                  
Validation: 100%|████████████| 164/164 [00:15<00:00, 10.89it/s, val_loss=0.0152]
Validation Loss: 0.0118, F1: 0.4493
Epoch [3/50], Train Loss: 0.2635, LR: 0.001000                                  
Validation: 100%|████████████| 164/164 [00:15<00:00, 10.88it/s, val_loss=0.0212]
Validation Loss: 0.0079, F1: 0.8422
Saved Best Model at epoch 3 with F1 score: 0.8422
Epoch [4/50], Train Loss: 0.2354, LR: 0.001000                                  
Validation: 100%|████████████| 164/164 [00:15<00:00, 10.88it/s, val_loss=0.0161]
Validation Loss: 0.0066, F1: 0.8856
Saved Best Model at epoch 4 with F1 score: 0.8856
Epoch [5/50], Train Loss: 0.2215, LR: 0.000100            

In [ ]:
# Test AML_Only model on AML Unseen Data

In [20]:
!python evaluate_vt.py --model_name vit_base_patch16_224 --data_dir AML_Unseen/ --model_path model_checkpoints/AML_Only/vit/vit_base_patch16_224_best.pth --num_classes 2 --batch_size 32

Evaluating: 100%|████████████████████| 29/29 [00:03<00:00,  9.59it/s, acc=0.944]

Evaluation Results:
Precision: 0.9090
Recall: 0.9174
F1: 0.9131
Accuracy: 0.9444
Balanced accuracy: 0.9174


In [ ]:
#-------- **************-----------------***********---

In [ ]:
#-------- **************-----------------***********---

In [ ]:
#-------- **************-----------------***********---

In [ ]:
#-------- **************-----------------***********---

In [ ]:
# Best AML model weights to TF on ALL data

In [ ]:
!python train-mobilenetv2.py --model_name mobilenetv2 --data_dir TLDataset/ALL/PKG-C-NMC2019 --num_classes 2 --num_epochs 150 --batch_size 64 --learning_rate 0.01 --momentum 0.9 --checkpoint_path model_checkpoints/AML_Pretrained/mobilenetv2 --pretrained_path model_checkpoints/AML_Only/mobilenetv2/mobilenetv2_best_epoch_104.pth

In [ ]:
# TF Model Validation on Unseen ALL

In [ ]:
!python evaluate-mobilenetv2.py --model_name mobilenetv2 --data_dir ALL_Unseen/ --model_path model_checkpoints/AML_Pretrained/mobilenetv2/mobilenetv2_best_epoch_100.pth --num_classes 2 --batch_size 32

In [ ]:
# TF Model Validation on Unseen AML

In [ ]:
!python evaluate.py --model_name resnet50 --data_dir AML_Unseen/ --model_path model_checkpoints/ALL_Pretrained/resnet50_best_epoch_76.pth --num_classes 2 --batch_size 32